In [29]:
pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
df = pd.read_csv(r"C:\Users\Sampath.emandi\Downloads\Complications_and_Deaths-Hospital.csv")

In [32]:
# Get a summary of all the columns and their data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90801 entries, 0 to 90800
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Facility ID           90801 non-null  object
 1   Facility Name         90801 non-null  object
 2   Address               90801 non-null  object
 3   City/Town             90801 non-null  object
 4   State                 90801 non-null  object
 5   ZIP Code              90801 non-null  int64 
 6   County/Parish         90801 non-null  object
 7   Telephone Number      90801 non-null  object
 8   Measure ID            90801 non-null  object
 9   Measure Name          90801 non-null  object
 10  Compared to National  90801 non-null  object
 11  Denominator           90801 non-null  object
 12  Score                 90801 non-null  object
 13  Lower Estimate        90801 non-null  object
 14  Higher Estimate       90801 non-null  object
 15  Footnote              42363 non-null

In [33]:
#  Get the names of all columns
df.columns


Index(['Facility ID', 'Facility Name', 'Address', 'City/Town', 'State',
       'ZIP Code', 'County/Parish', 'Telephone Number', 'Measure ID',
       'Measure Name', 'Compared to National', 'Denominator', 'Score',
       'Lower Estimate', 'Higher Estimate', 'Footnote', 'Start Date',
       'End Date'],
      dtype='object')

In [34]:
df = df.convert_dtypes()
# Convert columns to float or int
columns_to_convert = ['Denominator', 'Score', 'Lower Estimate', 'Higher Estimate']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
# General overview of the dataset
print("Data Info:")
df.info()

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90801 entries, 0 to 90800
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Facility ID           90801 non-null  string 
 1   Facility Name         90801 non-null  string 
 2   Address               90801 non-null  string 
 3   City/Town             90801 non-null  string 
 4   State                 90801 non-null  string 
 5   ZIP Code              90801 non-null  Int64  
 6   County/Parish         90801 non-null  string 
 7   Telephone Number      90801 non-null  string 
 8   Measure ID            90801 non-null  string 
 9   Measure Name          90801 non-null  string 
 10  Compared to National  90801 non-null  string 
 11  Denominator           45970 non-null  Int64  
 12  Score                 48934 non-null  Float64
 13  Lower Estimate        48934 non-null  Float64
 14  Higher Estimate       48934 non-null  Float64
 15  Footnote

In [35]:
# Check summary statistics for the Score column (e.g., mean, median, etc.)
df['Score'].describe()


count      48934.0
mean     11.196957
std      31.140403
min           0.05
25%           0.94
50%            3.1
75%           11.1
max         258.28
Name: Score, dtype: Float64

In [36]:
# Check the distribution of the "Compared to National" column
df['Compared to National'].value_counts()


Compared to National
No Different Than the National Rate     43940
Not Available                           30267
Number of Cases Too Small               11600
No Different Than the National Value     2708
Better Than the National Rate            1018
Worse Than the National Rate             1012
Worse Than the National Value             171
Better Than the National Value             85
Name: count, dtype: Int64

In [37]:
# Check the number of rows and columns
df.shape


(90801, 18)

In [38]:
print("\nColumns in the dataset:")
print(df.columns)


Columns in the dataset:
Index(['Facility ID', 'Facility Name', 'Address', 'City/Town', 'State',
       'ZIP Code', 'County/Parish', 'Telephone Number', 'Measure ID',
       'Measure Name', 'Compared to National', 'Denominator', 'Score',
       'Lower Estimate', 'Higher Estimate', 'Footnote', 'Start Date',
       'End Date'],
      dtype='object')


In [39]:
df.head(5)

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Measure ID,Measure Name,Compared to National,Denominator,Score,Lower Estimate,Higher Estimate,Footnote,Start Date,End Date
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,COMP_HIP_KNEE,Rate of complications for hip/knee replacement...,No Different Than the National Rate,32,3.0,1.6,5.5,<NA>,07/01/2020,03/31/2023
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_AMI,Death rate for heart attack patients,No Different Than the National Rate,291,10.8,8.4,13.6,<NA>,07/01/2020,06/30/2023
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_CABG,Death rate for CABG surgery patients,No Different Than the National Rate,157,3.8,2.1,6.8,<NA>,07/01/2020,06/30/2023
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_COPD,Death rate for COPD patients,No Different Than the National Rate,122,7.9,5.2,11.9,<NA>,07/01/2020,06/30/2023
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_HF,Death rate for heart failure patients,No Different Than the National Rate,610,10.5,8.6,12.9,<NA>,07/01/2020,06/30/2023


In [40]:
# Check for missing values
print("\nMissing values in each column:")
print(df.isnull().sum())


Missing values in each column:
Facility ID                 0
Facility Name               0
Address                     0
City/Town                   0
State                       0
ZIP Code                    0
County/Parish               0
Telephone Number            0
Measure ID                  0
Measure Name                0
Compared to National        0
Denominator             44831
Score                   41867
Lower Estimate          41867
Higher Estimate         41867
Footnote                48438
Start Date                  0
End Date                    0
dtype: int64


In [41]:
# Get a summary of the 'Score' column
print("\nSummary Statistics for 'Score':")
print(df['Score'].describe())


Summary Statistics for 'Score':
count      48934.0
mean     11.196957
std      31.140403
min           0.05
25%           0.94
50%            3.1
75%           11.1
max         258.28
Name: Score, dtype: Float64


In [42]:
# Get a summary of the 'Score' column
print("\nSummary Statistics:")
df.describe()


Summary Statistics:


,ZIP Code,Denominator,Score,Lower Estimate,Higher Estimate
count,90801.0,45970.0,48934.0,48934.0,48934.0
mean,54069.101904,1181.803285,11.196957,7.418498,15.374961
std,27003.18474,2541.444624,31.140403,22.812134,39.656359
min,603.0,25.0,0.05,0.0,0.25
25%,32955.0,90.0,0.94,0.07,1.72
50%,55417.0,269.0,3.1,0.94,5.38
75%,76208.0,966.0,11.1,7.6,15.8
max,99929.0,57005.0,258.28,220.22,302.62


In [43]:
# Check the distribution of the 'Compared to National' column (target variable)
print("\nDistribution of 'Compared to National':")
print(df['Compared to National'].value_counts())


Distribution of 'Compared to National':
Compared to National
No Different Than the National Rate     43940
Not Available                           30267
Number of Cases Too Small               11600
No Different Than the National Value     2708
Better Than the National Rate            1018
Worse Than the National Rate             1012
Worse Than the National Value             171
Better Than the National Value             85
Name: count, dtype: Int64


In [44]:
# Get the unique values in 'State'
print("\nUnique States:")
print(df['State'].unique())



Unique States:
<StringArray>
['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID',
 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO',
 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA',
 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VI', 'VA', 'WA', 'WV', 'WI',
 'WY', 'AS', 'GU', 'MP']
Length: 56, dtype: string


In [45]:
# Handle datetime columns (if necessary)
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])
# Check the date range
print("\nDate Range of 'Start Date':")
print(df['Start Date'].min(), df['Start Date'].max())


Date Range of 'Start Date':
2020-07-01 00:00:00 2021-07-01 00:00:00



### Data Preprocessing
#### Imputation of Missing Values
##### The dataset contains missing values in the following columns:
- Denominator (45970 non-null out of 90801)
- Score (48934 non-null out of 90801)
- Lower Estimate (48934 non-null out of 90801)
- Higher Estimate (48934 non-null out of 90801)
- Footnote (42363 non-null out of 90801)

Plan for Imputation:
1. For numerical columns (Denominator, Score, Lower Estimate, Higher Estimate):
        Use mean, median, or mode for imputation based on the distribution.
        Alternatively, use KNN imputation or regression-based methods if needed.
2. For the categorical column (Footnote):
        Use mode or forward/backward fill (if categorical continuity exists).
        For complex cases, treat missing values as a separate category.


In [46]:
df['Footnote'].value_counts()

Footnote
13       16332
1        12491
5         8718
19        2508
7         1761
28         480
1, 28       51
23          16
7, 28        5
1, 23        1
Name: count, dtype: Int64

In [47]:
# Categorical column imputation (mode strategy)
# Replace pd.NA with np.nan
df['Footnote'] = df['Footnote'].replace({pd.NA: np.nan})

In [48]:
# Numerical columns imputation (mean strategy)
num_imputer = SimpleImputer(strategy='mean')
df[['Denominator', 'Score', 'Lower Estimate', 'Higher Estimate']] = num_imputer.fit_transform(
    df[['Denominator', 'Score', 'Lower Estimate', 'Higher Estimate']]
)

In [49]:
df['Footnote'].replace({pd.NA: df['Footnote'].mode()[0]},inplace=True)

C:\Users\Sampath.emandi\AppData\Local\Temp\ipykernel_16184\1053024160.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Footnote'].replace({pd.NA: df['Footnote'].mode()[0]},inplace=True)


EDA

### Feature Engineering
Feature engineering creates new features or transforms existing ones to improve the model's predictive power.
#### Planned Transformations:
- Datetime Features:
   - Extract year, month, and duration (in days) from Start Date and End Date.
- Binning:
   - Create bins for Score, Lower Estimate, and Higher Estimate to categorize performance levels.
- Interaction Features:
   - Combine State, City/Town, and County/Parish into a single feature to represent a geographical hierarchy.
- Text Encoding:
   - Encode categorical columns (Compared to National, Footnote) using label encoding or one-hot encoding.

In [50]:
# Datetime feature engineering
df['Start_Year'] = df['Start Date'].dt.year
df['End_Year'] = df['End Date'].dt.year
df['Duration_Days'] = (df['End Date'] - df['Start Date']).dt.days

In [51]:
# Convert categorical columns to numerical using one-hot encoding (if necessary)
df = pd.get_dummies(df, drop_first=True)

# Split data into features (X) and target (y)
X = df.drop('Score', axis=1)
y = df['Score']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [52]:
X_train

,ZIP Code,Denominator,Lower Estimate,Higher Estimate,Start Date,End Date,Start_Year,End_Year,Duration_Days,Facility ID_010005,...,Compared to National_Worse Than the National Value,"Footnote_1, 23","Footnote_1, 28",Footnote_13,Footnote_19,Footnote_23,Footnote_28,Footnote_5,Footnote_7,"Footnote_7, 28"
30576,67860,30.0,13.00,27.40,2020-07-01,2023-06-30,2020,2023,1094,False,...,False,False,False,True,False,False,False,False,False,False
5790,95361,28.0,1.00,6.71,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
35274,70535,681.0,0.07,0.50,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
39493,49684,2988.0,1.52,4.21,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
87164,53121,169.0,0.41,4.30,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,90404,2107.0,3.03,6.74,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
54886,10468,440.0,1.55,6.79,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
76820,75961,45.0,5.10,12.40,2020-07-01,2023-06-30,2020,2023,1094,False,...,False,False,False,True,False,False,False,False,False,False
860,35401,610.0,14.60,20.50,2020-07-01,2023-06-30,2020,2023,1094,False,...,False,False,False,True,False,False,False,False,False,False


In [25]:
X_test

,ZIP Code,Denominator,Lower Estimate,Higher Estimate,Start Date,End Date,Start_Year,End_Year,Duration_Days,Facility ID_010005,...,Compared to National_Worse Than the National Value,"Footnote_1, 23","Footnote_1, 28",Footnote_13,Footnote_19,Footnote_23,Footnote_28,Footnote_5,Footnote_7,"Footnote_7, 28"
23405,60453,650.000000,0.570000,3.610000,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
47927,59538,1181.803285,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
20465,96720,1181.803285,7.418498,15.374961,2020-07-01,2023-06-30,2020,2023,1094,False,...,False,False,False,False,False,False,False,True,False,False
56146,11691,35.000000,9.500000,16.300000,2020-07-01,2023-06-30,2020,2023,1094,False,...,False,False,False,True,False,False,False,False,False,False
32340,40202,128.000000,1.100000,4.000000,2020-07-01,2023-06-30,2020,2023,1094,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26902,46975,1181.803285,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
24639,60033,1181.803285,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
9355,92870,975.000000,0.000000,1.450000,2021-07-01,2023-06-30,2021,2023,729,False,...,False,False,False,True,False,False,False,False,False,False
52026,7701,417.000000,8.800000,13.400000,2020-07-01,2023-06-30,2020,2023,1094,False,...,False,False,False,True,False,False,False,False,False,False


In [ ]:
# Calculate the duration between Start Date and End Date
X_train['duration'] = (X_train['End Date'] - X_train['Start Date']).dt.days
X_test['duration'] = (X_test['End Date'] - X_test['Start Date']).dt.days

# Now drop the original date columns
X_train = X_train.drop(['Start Date', 'End Date'], axis=1)
X_test = X_test.drop(['Start Date', 'End Date'], axis=1)

# Scale the data (including the 'duration' column now)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

2. Handling Imbalanced Data using Alternative Resampling Techniques
Instead of SMOTE, you can try undersampling, oversampling, or Hybrid Methods to address class imbalance.

Undersampling the Majority Class
In this technique, we reduce the number of instances from the majority class to balance the dataset.

In [ ]:


from imblearn.under_sampling import RandomUnderSampler

# Initialize RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Apply undersampling
X_resampled, y_resampled = undersampler.fit_resample(X_train_scaled, y_train)

# Check the distribution of the resampled data
print("Resampled class distribution:", pd.Series(y_resampled).value_counts())

,ZIP Code,Denominator,Score,Lower Estimate,Higher Estimate,Start Date,End Date,Start_Year,End_Year,Duration_Days,...,"Footnote_1, 23","Footnote_1, 28",Footnote_13,Footnote_19,Footnote_23,Footnote_28,Footnote_5,Footnote_7,"Footnote_7, 28",Score_to_Denominator
0,36301,32.000000,3.000000,1.600000,5.500000,2020-07-01,2023-03-31,2020,2023,1003,...,False,False,True,False,False,False,False,False,False,0.093750
1,36301,291.000000,10.800000,8.400000,13.600000,2020-07-01,2023-06-30,2020,2023,1094,...,False,False,True,False,False,False,False,False,False,0.037113
2,36301,157.000000,3.800000,2.100000,6.800000,2020-07-01,2023-06-30,2020,2023,1094,...,False,False,True,False,False,False,False,False,False,0.024204
3,36301,122.000000,7.900000,5.200000,11.900000,2020-07-01,2023-06-30,2020,2023,1094,...,False,False,True,False,False,False,False,False,False,0.064754
4,36301,610.000000,10.500000,8.600000,12.900000,2020-07-01,2023-06-30,2020,2023,1094,...,False,False,True,False,False,False,False,False,False,0.017213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90796,79014,1181.803285,11.196957,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,729,...,False,False,False,False,False,False,True,False,False,0.009474
90797,79014,1181.803285,11.196957,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,729,...,False,False,False,False,False,False,True,False,False,0.009474
90798,79014,1181.803285,11.196957,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,729,...,False,False,False,False,False,False,True,False,False,0.009474
90799,79014,1181.803285,11.196957,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,729,...,False,False,False,False,False,False,True,False,False,0.009474


#### 4. Addressing Imbalanced Data and Resampling Techniques
Imbalanced data can lead to biased predictions toward the majority class.

Steps:
Analyze Class Distribution:

Use value_counts() or visualizations like bar charts to check the imbalance.
Resampling Techniques:

a. Oversampling:

Use SMOTE (Synthetic Minority Oversampling Technique) to generate synthetic samples for the minority class.

In [28]:
# Assuming 'df' is your DataFrame and 'Target_Column' is the column name for the target
X = df.drop('Score', axis=1)  # Drop the target column to get features
y = df['Score']  # Select the target column


In [30]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (90801, 22)
Shape of y: (90801,)


In [31]:
# Example: One-hot encoding for categorical variables
X = pd.get_dummies(X, drop_first=True)


In [34]:
X

,ZIP Code,Measure Name,Compared to National,Denominator,Lower Estimate,Higher Estimate,Start Date,End Date,Start_Year,End_Year,...,Measure ID_PSI_90,"Footnote_1, 23","Footnote_1, 28",Footnote_13,Footnote_19,Footnote_23,Footnote_28,Footnote_5,Footnote_7,"Footnote_7, 28"
0,36301,18,2,32.000000,1.600000,5.500000,2020-07-01,2023-03-31,2020,2023,...,False,False,False,True,False,False,False,False,False,False
1,36301,5,2,291.000000,8.400000,13.600000,2020-07-01,2023-06-30,2020,2023,...,False,False,False,True,False,False,False,False,False,False
2,36301,3,2,157.000000,2.100000,6.800000,2020-07-01,2023-06-30,2020,2023,...,False,False,False,True,False,False,False,False,False,False
3,36301,4,2,122.000000,5.200000,11.900000,2020-07-01,2023-06-30,2020,2023,...,False,False,False,True,False,False,False,False,False,False
4,36301,6,2,610.000000,8.600000,12.900000,2020-07-01,2023-06-30,2020,2023,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90796,79014,11,4,1181.803285,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,...,False,False,False,False,False,False,False,True,False,False
90797,79014,15,4,1181.803285,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,...,False,False,False,False,False,False,False,True,False,False
90798,79014,16,4,1181.803285,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,...,False,False,False,False,False,False,False,True,False,False
90799,79014,0,4,1181.803285,7.418498,15.374961,2021-07-01,2023-06-30,2021,2023,...,False,False,False,False,False,False,False,True,False,False


In [52]:
from imblearn.under_sampling import RandomUnderSampler

# Initialize RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Apply undersampling
X_resampled, y_resampled = undersampler.fit_resample(X_train_scaled, y_train)

# Check the distribution of the resampled data
print("Resampled class distribution:", pd.Series(y_resampled).value_counts())


ModuleNotFoundError: No module named 'imblearn'